!pip install BeautifulSoup4, lxml, pandas, requests, IPython, googlemaps

In [ ]:
import json, config
from urllib.request import urlopen
from bs4 import BeautifulSoup
import googlemaps

In [ ]:
gmaps = googlemaps.Client(key=config.google_api_key)

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_current_National_Football_League_stadiums"

In [ ]:
page = urlopen(url)

In [ ]:
soup = BeautifulSoup(page, "lxml")

In [ ]:
first_table = soup.find('table', class_='wikitable sortable plainrowheaders')

In [ ]:
A=[] #IMG
B=[] #Arena
C=[] #Capacity
D=[] #City
E=[] #surface
F=[] #Roof Type
G=[] #Team(s)
H=[] #Opened
I=[] #State

In [ ]:
for row in first_table.findAll('tr'):
    cells = row.findAll(['td', 'th'])
    if cells[0].find(scope_=lambda x: x != 'col'):

        A.append(cells[0].find('img')['src'].replace("120px", "250px"))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True).rstrip('\n'))
        ##this splits the city and state then puts them in their own arrays.
        location = cells[3].find(text=True).split(',')
        
        D.append(location[0])
        I.append(location[1].lstrip())
        E.append(cells[4].find(text=True).rstrip('\n'))
        F.append(cells[5].find(text=True).rstrip('\n'))
        teams = cells[6].findAll('a')
        if len(teams) > 1 :
            tempArray= []
            for i in teams:
                tempArray.append(str(i.find_all(text=True)).rstrip('[')[2:len(i)-3])
            G.append(" and ".join(tempArray))
        else:
            G.append(cells[6].find(text=True))
        H.append(cells[7].find(text=True).rstrip('\n'))


In [ ]:
import pandas as pd
df=pd.DataFrame(G,columns=['Team'])
df['Arena']= B
df['City']= D
df['State'] = I
df['Capacity']= C
df['opened']= H
df['surface']= E
df['Roof Type']= F
df['image']= A
df

In [ ]:
location = [];
for i in range(len(B)):
    searchTerm = B[i] + ", " + D[i] + "," + I[i]
    geocode_result = gmaps.geocode(searchTerm)
    data = geocode_result[0]['geometry']['location']
    tempArray = []
    tempArray.append(float(data['lng']))
    tempArray.append(float(data['lat']))
    location.append(tempArray[::-1])


In [ ]:
final_data = []
for i in range(len(A)):
    temp={}
    temp['Image'] = "HTTPS:" + A[i]
    temp['Team'] = G[i]
    temp['Arena'] = B[i]
    temp['Location'] = D[i] + ", " + I[i]
    temp['Surface'] = E[i]
    temp['Roof Type'] = F[i]
    temp['Capacity'] = C[i]
    temp['opened'] = H[i]
    temp['Coordinates'] = location[i]
    final_data.append(temp)

In [ ]:
print(len(final_data))

In [ ]:
with open('NFL.json', 'w') as outfile:
    json.dump(final_data, outfile)

In [ ]:
import requests, IPython.display as Disp
url = final_data[8]['Image']
Disp.Image(requests.get(url).content)